## Imports

In [54]:
import copy
import gurobipy as gp
import json
import math
import numpy as np
import os
import pandas as pd
import random
from gurobipy import GRB
from pprint import pprint

## Read Data

In [2]:
TRANSFER_PERCENTAGE = 0
# TRANSFER_PERCENTAGE = 0.25

if TRANSFER_PERCENTAGE > 0:
    INPUT_PATH = f"data/boston-logan/transfers/{TRANSFER_PERCENTAGE}"
else:
    INPUT_PATH = "data/boston-logan"

arrivals_df = pd.read_csv(os.path.join(INPUT_PATH, "arrivals.csv"))
departures_df = pd.read_csv(os.path.join(INPUT_PATH, "departures.csv"))
gates_df = pd.read_excel("data/boston-logan/Gates.xlsx")

if TRANSFER_PERCENTAGE > 0:
    with open(os.path.join(INPUT_PATH, "transfers.json")) as json_file:
        transfers = json.load(json_file)

all_flights_df = pd.concat([arrivals_df, departures_df])

In [3]:
arrivals_df

,Unnamed: 0,Flight Number,Arrival Time,Departure Time,Passengers,Type
0,0,2,25,136,106,Arrival
1,1,3,41,161,147,Arrival
2,2,6,73,187,187,Arrival
3,3,7,77,188,121,Arrival
4,4,10,111,233,94,Arrival
...,...,...,...,...,...,...
63,63,120,1325,1447,127,Arrival
64,64,124,1369,1499,88,Arrival
65,65,126,1390,1513,240,Arrival
66,66,127,1398,1513,87,Arrival


In [4]:
gates_df ## Gates Start from 1 (Gate 0 is the terminal entry)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,53,45,43,31,29,25,23,19,9,3,15,17,21,23
1,1,53,0,4,10,22,24,28,30,34,44,50,62,64,68,70
2,2,45,4,0,6,18,20,24,26,30,40,46,58,60,64,66
3,3,43,10,6,0,12,14,18,20,24,34,40,52,54,58,60
4,4,31,22,18,12,0,2,6,8,12,22,28,40,42,46,48
5,5,29,24,20,14,2,0,4,6,10,20,26,38,40,44,46
6,6,25,28,24,18,6,4,0,2,6,16,22,34,36,40,42
7,7,23,30,26,20,8,6,2,0,4,14,20,32,34,38,40
8,8,19,34,30,24,12,10,6,4,0,10,16,28,30,34,36
9,9,9,44,40,34,22,20,16,14,10,0,6,18,20,24,26


## Utility Functions

In [5]:
def get_distance(i, j):
    return gates_df[i][j]

def get_arrival_time(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Arrival Time"]
    assert(len(result) == 1)
    return int(result)

def get_departure_time(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Departure Time"]
    assert(len(result) == 1)
    return int(result)

def get_num_passengers(i):
    result = all_flights_df[all_flights_df["Flight Number"] == i]["Passengers"]
    assert(len(result) == 1)
    return int(result)

### Tests

In [6]:
assert(get_distance(5,3) == 14)
assert(get_distance(14,14) == 0)
assert(get_distance(4,11) == 40)

In [7]:
## Basic Assertions on the Data
flight_numbers = sorted(list(arrivals_df["Flight Number"]) + list(departures_df["Flight Number"]))
assert(flight_numbers == list(range(1,131)))

flight_numbers = sorted(list(all_flights_df["Flight Number"]))
assert(flight_numbers == list(range(1,131)))

## Parameters

In [8]:
num_flights = 130
num_gates = 14

## Descriptors (for variables)

In [9]:
F = [f"Flight_{i}" for i in range(1, num_flights + 1)]
F

['Flight_1',
 'Flight_2',
 'Flight_3',
 'Flight_4',
 'Flight_5',
 'Flight_6',
 'Flight_7',
 'Flight_8',
 'Flight_9',
 'Flight_10',
 'Flight_11',
 'Flight_12',
 'Flight_13',
 'Flight_14',
 'Flight_15',
 'Flight_16',
 'Flight_17',
 'Flight_18',
 'Flight_19',
 'Flight_20',
 'Flight_21',
 'Flight_22',
 'Flight_23',
 'Flight_24',
 'Flight_25',
 'Flight_26',
 'Flight_27',
 'Flight_28',
 'Flight_29',
 'Flight_30',
 'Flight_31',
 'Flight_32',
 'Flight_33',
 'Flight_34',
 'Flight_35',
 'Flight_36',
 'Flight_37',
 'Flight_38',
 'Flight_39',
 'Flight_40',
 'Flight_41',
 'Flight_42',
 'Flight_43',
 'Flight_44',
 'Flight_45',
 'Flight_46',
 'Flight_47',
 'Flight_48',
 'Flight_49',
 'Flight_50',
 'Flight_51',
 'Flight_52',
 'Flight_53',
 'Flight_54',
 'Flight_55',
 'Flight_56',
 'Flight_57',
 'Flight_58',
 'Flight_59',
 'Flight_60',
 'Flight_61',
 'Flight_62',
 'Flight_63',
 'Flight_64',
 'Flight_65',
 'Flight_66',
 'Flight_67',
 'Flight_68',
 'Flight_69',
 'Flight_70',
 'Flight_71',
 'Flight_72',
 

In [10]:
G = [f"Gate_{i}" for i in range(1, num_gates + 1)]
G

['Gate_1',
 'Gate_2',
 'Gate_3',
 'Gate_4',
 'Gate_5',
 'Gate_6',
 'Gate_7',
 'Gate_8',
 'Gate_9',
 'Gate_10',
 'Gate_11',
 'Gate_12',
 'Gate_13',
 'Gate_14']

## Greedy Solution

In [11]:
sorted_flights = all_flights_df.sort_values("Departure Time", ascending=True)

In [35]:
def greedy_solution(num_flights, num_gates, sorted_flights):
    gates = [[] for i in range(num_gates)]
    for i in range(num_gates):
        for j in np.arange(int(num_flights / num_gates)):
            gates[i].append(sorted_flights.iloc[j*num_gates + i]["Flight Number"])

    remaining = num_flights - (num_gates * int(num_flights / num_gates))
    for i in range(remaining):
        gates[i].append(sorted_flights.iloc[num_gates * int(num_flights / num_gates) + i]["Flight Number"])
    return gates

def get_assignment(gates):
    assignment = dict()
    for j, gate_j in enumerate(gates):
        for flight_i in gate_j:
            g = j + 1
            assignment[flight_i] = g
    return assignment

def get_X_matrix(num_flights, num_gates, gates):
    X = np.zeros((num_flights, num_gates))
    for j, gate_j in enumerate(gates):
        for flight_i in gate_j:
            X[(flight_i - 1)][j] = 1
    return X

def check_constraints(gates):
    for j, gate_j in enumerate(gates):
        for flight_i in gate_j:
            ai, di = get_arrival_time(flight_i), get_departure_time(flight_i)
            for flight_j in gate_j:
                if flight_i == flight_j:
                    continue
                aj, dj = get_arrival_time(flight_j), get_departure_time(flight_j)
                if (dj - ai) * (di - aj) > 0:
                    return False
    return True

def check_constraints(assignment):
    for flight_i in assignment.keys():
        ai, di = get_arrival_time(flight_i), get_departure_time(flight_i)
        for flight_j in assignment.keys():
            if flight_i == flight_j or assignment[flight_i] != assignment[flight_j]:
                continue
            aj, dj = get_arrival_time(flight_j), get_departure_time(flight_j)
            if (dj - ai) * (di - aj) > 0:
                return False
    return True
    

def get_objective_value(assignment):
    dep_arr_cost = 0
    transfer_cost = 0
    for i in assignment.keys():
        g = assignment[i]
        dep_arr_cost += get_num_passengers(i) * get_distance(0, g)

    if TRANSFER_PERCENTAGE > 0:
        print("Adding Transfer Distances to Objective Function...")
        for i in assignment.keys():
            i_key = str(i)
            k = assignment[i]
            if i_key in transfers:
                transfers_flight_i = transfers[i_key]
                for j_key in transfers_flight_i:
                    j = int(j_key)
                    l = assignment[j]
                    pij = transfers[i_key][j_key]
                    wkl = get_distance(k, l)
                    transfer_cost += pij * wkl
    objective = dep_arr_cost + transfer_cost               
    return (dep_arr_cost, transfer_cost, objective)

In [36]:
gates = greedy_solution(num_flights, num_gates, sorted_flights)
X = get_X_matrix(num_flights, num_gates, gates)
assignment = get_assignment(gates)
(dep_arr_cost, transfer_cost, objective) = get_objective_value(assignment)

In [37]:
gates

[[1, 15, 29, 43, 57, 71, 84, 99, 113, 126],
 [2, 16, 30, 44, 58, 72, 87, 101, 114, 128],
 [3, 17, 31, 45, 59, 73, 86, 100, 115, 129],
 [4, 18, 32, 47, 61, 74, 89, 102, 116, 130],
 [5, 19, 34, 46, 60, 75, 88, 103, 117],
 [6, 20, 33, 48, 62, 77, 90, 104, 119],
 [7, 21, 35, 50, 63, 76, 91, 105, 118],
 [9, 22, 36, 49, 64, 78, 92, 107, 120],
 [8, 23, 37, 51, 65, 79, 93, 106, 121],
 [10, 24, 38, 52, 67, 80, 94, 108, 122],
 [11, 25, 39, 53, 66, 81, 95, 109, 123],
 [12, 26, 40, 54, 68, 82, 96, 110, 125],
 [13, 27, 41, 55, 69, 83, 97, 111, 124],
 [14, 28, 42, 56, 70, 85, 98, 112, 127]]

In [38]:
X

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
assignment

{1: 1,
 15: 1,
 29: 1,
 43: 1,
 57: 1,
 71: 1,
 84: 1,
 99: 1,
 113: 1,
 126: 1,
 2: 2,
 16: 2,
 30: 2,
 44: 2,
 58: 2,
 72: 2,
 87: 2,
 101: 2,
 114: 2,
 128: 2,
 3: 3,
 17: 3,
 31: 3,
 45: 3,
 59: 3,
 73: 3,
 86: 3,
 100: 3,
 115: 3,
 129: 3,
 4: 4,
 18: 4,
 32: 4,
 47: 4,
 61: 4,
 74: 4,
 89: 4,
 102: 4,
 116: 4,
 130: 4,
 5: 5,
 19: 5,
 34: 5,
 46: 5,
 60: 5,
 75: 5,
 88: 5,
 103: 5,
 117: 5,
 6: 6,
 20: 6,
 33: 6,
 48: 6,
 62: 6,
 77: 6,
 90: 6,
 104: 6,
 119: 6,
 7: 7,
 21: 7,
 35: 7,
 50: 7,
 63: 7,
 76: 7,
 91: 7,
 105: 7,
 118: 7,
 9: 8,
 22: 8,
 36: 8,
 49: 8,
 64: 8,
 78: 8,
 92: 8,
 107: 8,
 120: 8,
 8: 9,
 23: 9,
 37: 9,
 51: 9,
 65: 9,
 79: 9,
 93: 9,
 106: 9,
 121: 9,
 10: 10,
 24: 10,
 38: 10,
 52: 10,
 67: 10,
 80: 10,
 94: 10,
 108: 10,
 122: 10,
 11: 11,
 25: 11,
 39: 11,
 53: 11,
 66: 11,
 81: 11,
 95: 11,
 109: 11,
 123: 11,
 12: 12,
 26: 12,
 40: 12,
 54: 12,
 68: 12,
 82: 12,
 96: 12,
 110: 12,
 125: 12,
 13: 13,
 27: 13,
 41: 13,
 55: 13,
 69: 13,
 83: 13,
 97: 

In [41]:
# assert(check_constraints(gates))
assert(check_constraints(assignment))

In [42]:
print((dep_arr_cost, transfer_cost, objective))

(561116, 0, 561116)


## Simulated Annealing Algorithm

In [43]:
transfers

NameError: name 'transfers' is not defined

In [57]:
def get_neighbor(assignment):
    assignment = copy.copy(assignment)

    def swap(n):
        i = random.sample(range(1, n + 1), 2)
        i1, i2 = i[0], i[1]
        g1 = assignment[i1]
        g2 = assignment[i2]
        # print(i1, g1, i2, g2)
        assignment[i1] = g2
        assignment[i2] = g1

    def insert(n, g):
        i1 = random.sample(range(1, n + 1), 1)
        g = random.sample(range(1, g + 1), 1)
        # print(i1, g)
        assignment[i1[0]] = g[0]
    
    s = random.choice([1,2])
    if s == 1:
        swap(num_flights)
    elif s == 2:
        insert(num_flights, num_gates)
    return assignment

assignment2 = create_neighbor(assignment)

96 12 29 1


In [58]:
pprint(assignment)

{1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 9,
 9: 8,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 1,
 16: 2,
 17: 3,
 18: 4,
 19: 5,
 20: 6,
 21: 7,
 22: 8,
 23: 9,
 24: 10,
 25: 11,
 26: 12,
 27: 13,
 28: 14,
 29: 1,
 30: 2,
 31: 3,
 32: 4,
 33: 6,
 34: 5,
 35: 7,
 36: 8,
 37: 9,
 38: 10,
 39: 11,
 40: 12,
 41: 13,
 42: 14,
 43: 1,
 44: 2,
 45: 3,
 46: 5,
 47: 4,
 48: 6,
 49: 8,
 50: 7,
 51: 9,
 52: 10,
 53: 11,
 54: 12,
 55: 13,
 56: 14,
 57: 1,
 58: 2,
 59: 3,
 60: 5,
 61: 4,
 62: 6,
 63: 7,
 64: 8,
 65: 9,
 66: 11,
 67: 10,
 68: 12,
 69: 13,
 70: 14,
 71: 1,
 72: 2,
 73: 3,
 74: 4,
 75: 5,
 76: 7,
 77: 6,
 78: 8,
 79: 9,
 80: 10,
 81: 11,
 82: 12,
 83: 13,
 84: 1,
 85: 14,
 86: 3,
 87: 2,
 88: 5,
 89: 4,
 90: 6,
 91: 7,
 92: 8,
 93: 9,
 94: 10,
 95: 11,
 96: 12,
 97: 13,
 98: 14,
 99: 1,
 100: 3,
 101: 2,
 102: 4,
 103: 5,
 104: 6,
 105: 7,
 106: 9,
 107: 8,
 108: 10,
 109: 11,
 110: 12,
 111: 13,
 112: 14,
 113: 1,
 114: 2,
 115: 3,
 116: 4,
 117: 5,
 118: 7,
 119: 

In [59]:
pprint(assignment2)

{1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 9,
 9: 8,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 1,
 16: 2,
 17: 3,
 18: 4,
 19: 5,
 20: 6,
 21: 7,
 22: 8,
 23: 9,
 24: 10,
 25: 11,
 26: 12,
 27: 13,
 28: 14,
 29: 12,
 30: 2,
 31: 3,
 32: 4,
 33: 6,
 34: 5,
 35: 7,
 36: 8,
 37: 9,
 38: 10,
 39: 11,
 40: 12,
 41: 13,
 42: 14,
 43: 1,
 44: 2,
 45: 3,
 46: 5,
 47: 4,
 48: 6,
 49: 8,
 50: 7,
 51: 9,
 52: 10,
 53: 11,
 54: 12,
 55: 13,
 56: 14,
 57: 1,
 58: 2,
 59: 3,
 60: 5,
 61: 4,
 62: 6,
 63: 7,
 64: 8,
 65: 9,
 66: 11,
 67: 10,
 68: 12,
 69: 13,
 70: 14,
 71: 1,
 72: 2,
 73: 3,
 74: 4,
 75: 5,
 76: 7,
 77: 6,
 78: 8,
 79: 9,
 80: 10,
 81: 11,
 82: 12,
 83: 13,
 84: 1,
 85: 14,
 86: 3,
 87: 2,
 88: 5,
 89: 4,
 90: 6,
 91: 7,
 92: 8,
 93: 9,
 94: 10,
 95: 11,
 96: 1,
 97: 13,
 98: 14,
 99: 1,
 100: 3,
 101: 2,
 102: 4,
 103: 5,
 104: 6,
 105: 7,
 106: 9,
 107: 8,
 108: 10,
 109: 11,
 110: 12,
 111: 13,
 112: 14,
 113: 1,
 114: 2,
 115: 3,
 116: 4,
 117: 5,
 118: 7,
 119: 

In [60]:
def simulated_annealing(assignment, T=10, alpha=0.99):
    total_iterations = 100
    iterations_per_epoch = 10
    assignment = assignment
    _, _, value = get_objective_value(assignment)
    
    best_assignment = assignment
    best_value = value
    for i in range(total_iterations):
        print(f"Iteration {i}")
        print("Best Value: ", best_value)
        print("Value: ", value)
        
        neighbor_feasible = False
        while not neighbor_feasible:
            assignment_2 = get_neighbor(assignment)
            if check_constraints(assignment_2):
                neighbor_feasible = True
        
        assert(neighbor_feasible)
        _, _, value2 = get_objective_value(assignment_2)
        if value2 < value:
            assignment = assignment_2
            value = value2
        else:
            delta = value2 - value
            p = math.exp(-delta / T)
            rand = random.uniform(0, 1)
            
            if rand <= p:
                assignment = assignment_2
                value = value2
            
        if value < best_value:
            best_assignment = assignment
            best_value = value
        
        if iterations_per_epoch % 10 == 0:
            T = T * alpha

In [61]:
simulated_annealing(assignment)

Iteration 0
Best Value:  561116
Value:  561116
Iteration 1
Best Value:  561116
Value:  561116
Iteration 2
Best Value:  561116
Value:  561116
Iteration 3
Best Value:  561116
Value:  561116
Iteration 4
Best Value:  561116
Value:  561116
Iteration 5
Best Value:  561116
Value:  561116
Iteration 6
Best Value:  560712
Value:  560712
Iteration 7
Best Value:  560712
Value:  560712
Iteration 8
Best Value:  560598
Value:  560598
Iteration 9
Best Value:  560370
Value:  560370
Iteration 10
Best Value:  560370
Value:  560370
Iteration 11
Best Value:  560370
Value:  560370
Iteration 12
Best Value:  560370
Value:  560370
Iteration 13
Best Value:  560370
Value:  560370
Iteration 14
Best Value:  558038
Value:  558038
Iteration 15
Best Value:  558038
Value:  558038
Iteration 16
Best Value:  558038
Value:  558038
Iteration 17
Best Value:  557540
Value:  557540
Iteration 18
Best Value:  557540
Value:  557540
Iteration 19
Best Value:  557540
Value:  557540
Iteration 20
Best Value:  557540
Value:  557540
It